In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
print(net(X))

tensor([[-0.0408, -0.1318,  0.2015, -0.0984, -0.1089,  0.1365,  0.1671, -0.1466,
         -0.1232,  0.1052],
        [ 0.0257, -0.1761,  0.1947, -0.1014, -0.0435,  0.1751,  0.1118,  0.0095,
         -0.0634,  0.1716]], grad_fn=<AddmmBackward0>)


In [2]:
# 自定义块
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
print(net(X))

tensor([[-0.2648,  0.0152, -0.1055, -0.0868, -0.1441, -0.1813,  0.1311, -0.0499,
         -0.0980, -0.0625],
        [-0.3339,  0.0483, -0.0450,  0.0399, -0.0492, -0.2325, -0.0102, -0.1122,
          0.0249,  0.0316]], grad_fn=<AddmmBackward0>)


In [5]:
# 顺序块
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
print(net(X))

tensor([[-0.3810, -0.0785,  0.1672,  0.0300, -0.1075,  0.0133,  0.2384, -0.1365,
          0.1077, -0.0796],
        [-0.1226, -0.0961,  0.0913, -0.0420, -0.1777,  0.0386,  0.1378, -0.0488,
          0.1486,  0.0278]], grad_fn=<AddmmBackward0>)


In [10]:
# 再向前传播中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常数参数，以及用广播操作
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [11]:
net = FixedHiddenMLP()
print(net(X))

tensor(0.2456, grad_fn=<SumBackward0>)


In [12]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.3491, grad_fn=<SumBackward0>)